In [639]:
%%time
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
import tools.Sample_Tools as smpl
import Analysis_Funs as af
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import talib

from tools.Test_Tools import FactorTest
import tools.Pretreat_Tools as pretreat
from base.JuUnits import excute_for_multidates

import toys
# %load_ext autoreload
# %autoreload 2
# %aimport tools.Test_Tools,toys




# data = smpl.get_data(['000002'],end='2020-11-29',gap=528).data

# ttt =TTT(data)
# ttt.fit()

# fig = plt.figure(figsize=(1620/72,620/72))
# ax1= fig.add_subplot(2,1,1)
# # pd.concat([data.close,ht.ind_df['zhouqi']*5,ht.ind_df['jieduan']],axis=1).plot(secondary_y=['zhouqi','jieduan'],ax=ax1)
# # pd.concat([data.close,ht.ind_df['inphase'],ht.ind_df['quadrature']],axis=1).plot(secondary_y=['inphase','quadrature'],ax=ax1)
# pd.concat([data.close,ttt.ind_df['TTT']],axis=1).plot(secondary_y=['TTT'],ax=ax1)
# plt.grid()
# ax2= fig.add_subplot(6,1,4,sharex=ax1)
# data.volume.plot(kind='bar',ax=ax2)
# ax2.tick_params(labelbottom=True)  #  解决share后标签不可见的问题。
# ax2.xaxis.set_major_locator(ticker.MaxNLocator(6))



Wall time: 0 ns


In [675]:


data = smpl.get_sample_by_zs(name='上证50', end='2020-11-28', gap=150,  only_main=True)
returns = smpl.get_current_return(data,'close')
returns.name = 'returns'
ret_forward = smpl.get_forward_return(data,'close')
ret_forward.name = 'ret_forward'
# {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
data = pd.concat([data.data, returns, ret_forward], axis=1)
data = data.assign(vwap=data.amount/(data.volume*100))
data = smpl.add_marketvalue_industry(data)
data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)


close_ind = pretreat.neutralize(data.close, data['ind'],categorical=['ind'])
close_ind.name = 'close_ind'
vwap_ind = pretreat.neutralize(data.vwap, data['ind'],categorical=['ind'])
vwap_ind.name = 'vwap_ind'
low_ind = pretreat.neutralize(data.low, data['ind'],categorical=['ind'])
low_ind.name = 'low_ind'
volume_ind = pretreat.neutralize(data.volume, data['ind'],categorical=['ind'])
volume_ind.name = 'volume_ind'

adv20 = excute_for_multidates(data.volume, lambda x:x.rolling(20).agg('mean'), level=1)
adv20 = pd.concat([adv20,data['ind']],axis=1).dropna()
adv20_ind = pretreat.neutralize(adv20.volume, adv20['ind'],categorical=['ind'])
adv20_ind.name = 'adv20_ind'

adv40 = excute_for_multidates(data.volume, lambda x:x.rolling(40).agg('mean'), level=1)
adv40 = pd.concat([adv40, data['ind']],axis=1).dropna()
adv40_ind = pretreat.neutralize(adv40.volume, adv40['ind'],categorical=['ind'])
adv40_ind.name = 'adv40_ind'

adv81 = excute_for_multidates(data.volume, lambda x:x.rolling(81).agg('mean'), level=1)
adv81 = pd.concat([adv81, data['ind']],axis=1).dropna()
adv81_ind = pretreat.neutralize(adv81.volume, adv81['ind'],categorical=['ind'])
adv81_ind.name = 'adv81_ind'

co_mixed = ((data.close * 0.60733) + (data.Open * (1 - 0.60733)))
co_mixed_ind = pretreat.neutralize(co_mixed, data['ind'],categorical=['ind'])
co_mixed_ind.name = 'co_mixed_ind'

oh_mixed = ((data.Open * 0.868128) + (data.high * (1 - 0.868128)))
oh_mixed_ind = pretreat.neutralize(oh_mixed, data['ind'],categorical=['ind'])
oh_mixed_ind.name = 'oh_mixed_ind'

lv_mixed = ((data.low * 0.721001) + (data.vwap * (1 - 0.721001)))
lv_mixed_ind = pretreat.neutralize(lv_mixed, data['ind'],categorical=['ind'])
lv_mixed_ind.name = 'lv_mixed_ind'

data = pd.concat([data, close_ind, vwap_ind, low_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)
data

Open    high     low   close     volume        amount  \
date       code                                                              
2020-04-17 600000   10.18   10.23   10.15   10.20   227444.0  2.317890e+08   
           600028    4.50    4.58    4.48    4.56  2118188.0  9.614349e+08   
           600030   23.69   23.82   23.34   23.38  1504200.0  3.546707e+09   
           600031   19.36   19.92   19.18   19.60   787245.0  1.544122e+09   
           600036   32.45   33.20   32.34   32.86   697952.0  2.293965e+09   
...                   ...     ...     ...     ...        ...           ...   
2020-11-26 601995   59.56   62.79   59.18   62.20   492693.0  3.014638e+09   
           603259  103.70  104.18  101.92  103.00    95330.0  9.810765e+08   
           603288  159.35  161.27  159.35  160.05    18171.0  2.911082e+08   
           603501  209.80  209.85  202.28  202.77    62307.0  1.284851e+09   
           603986  205.80  207.55  201.49  201.76    57958.0  1.186319e+09   

                    returns  ret_forward        vwap        shares  ...  \
date       code                                                     ...   
2020-04-17 600000  0.003937    -0.008824   10.191036  2.935208e+06  ...   
           600028  0.020134     0.010965    4.538950  1.210712e+07  ...   
           600030 -0.012252     0.004705   23.578694  1.292678e+06  ...   
           600031  0.022965     0.001531   19.614256  8.435533e+05  ...   
           600036  0.023676     0.001217   32.867093  2.521984e+06  ...   
...                     ...          ...         ...           ...  ...   
2020-11-26 601995  0.033566     0.012058   61.186937  4.827257e+05  ...   
           603259  0.007828    -0.001650  102.913725  2.442021e+05  ...   
           603288  0.005339     0.011371  160.204827  3.240443e+05  ...   
           603501 -0.033969     0.014943  206.212916  8.635766e+04  ...   
           603986 -0.019631     0.009070  204.685909  4.716267e+04  ...   

                   close_ind   vwap_ind  low_ind volume_ind  adv20_ind  \
date       code                                                          
2020-04-17 600000   -11.1750 -11.121464 -10.9750  -508812.0        NaN   
           600028     0.2475   0.163950   0.1675   920108.0        NaN   
           600030    -1.9950  -1.858806  -1.4725   683976.0        NaN   
           600031    -2.7750  -2.823244  -2.8200    23853.0        NaN   
           600036    11.4850  11.554593  11.2150   -38304.0        NaN   
...                      ...        ...      ...        ...        ...   
2020-11-26 601995    36.8250  35.749437  34.3675  -327531.0        NaN   
           603259    -1.0625  -1.148775  -0.1425   -51614.0   -2974.45   
           603288    15.8625  16.142327  17.6000   -35077.0  -15690.10   
           603501    -1.7300   1.837916   2.6550   -26781.0   33462.00   
           603986    -2.7400   0.310909   1.8650   -31130.0   27821.80   

                   adv40_ind     adv81_ind  co_mixed_ind  oh_mixed_ind  \
date       code                                                          
2020-04-17 600000        NaN           NaN    -11.182853    -11.250906   
           600028        NaN           NaN      0.223940      0.198050   
           600030        NaN           NaN     -1.998272     -1.605357   
           600031        NaN           NaN     -2.869241     -2.941152   
           600036        NaN           NaN     11.324005     11.111404   
...                      ...           ...           ...           ...   
2020-11-26 601995        NaN           NaN     35.663351     34.673447   
           603259  -3377.975 -11817.728395     -0.787631     -0.486701   
           603288 -13739.800  -3842.049383     15.712631     15.415694   
           603501  11192.525   5505.851852      1.092970      4.744094   
           603986  -4074.300  -1650.259259     -1.091113      0.968276   

                   lv_mixed_ind  
date       code                  
2020-04-17 600000    -11.151051  
  

In [635]:
%%time

def check_quality(ind_binned_ret):
    x = ind_binned_ret
    # x = x/np.abs(x).max()+0.000001
    r_cor = np.corrcoef(x.rank(ascending=False),x.index)[0][1]
    # print(r_cor,x.idxmin(),x.idxmax())
    
    if np.abs(r_cor)>0.45:
        if x.idxmax() in [1,2]:
            return 'valid_left'
        if x.idxmax() in [9,10]:
            return 'valid_right'
        return 'invalid'
    
    if np.abs(r_cor) < 0.16:
        if x.idxmax() in [5,6]:
            return 'sem_mid'
        if x.idxmax() in [1,10]:
            x[x.idxmax()]=0
            if x.idxmax() in [1,10]:
                return 'sem_bilateral'
        return 'invalid'
    
    return 'invalid'

# fun_name = 'alpha101'
for idx in range(1,102):
    fun_name = 'alpha'+str(idx)
    print(fun_name)
    
    params = inspect.signature(getattr(toys, fun_name)).parameters.keys()
    if 'ind' in params:
        # raise 'pass' # 暂时先不处理中性化问题。
        indx = excute_for_multidates(data, lambda x: getattr(toys, fun_name)(*[x[param].copy() for param in params]) ,level=1)
    else:
        indx = excute_for_multidates(data, lambda x: getattr(toys, fun_name)(*[x[param].copy() for param in params]) ,level=1)
    indx.name = fun_name
    tes = FactorTest(target_field=fun_name)

    tes.process_ind(pd.DataFrame(indx),data.ret_forward,simple=True)
    try:
        x = tes.get_ind_binned_ret_avg()
        if len(x) == 0:
            print('errrrrrrorlen(x) == 0\r\n')
            continue
    except :
        with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.width', 999):
            print(tes.ind_binned)
        print('errrrrrror\r\n')
        continue

    quality = check_quality(x)
    print(quality)
    # tes.plot(only_binned=True)
    # print('\r\n')



# # tes.ind_binned
# # tes.ind_ret_df
# x = tes.ind_ret_df.groupby(level=0, group_keys=False).apply(lambda x: pretreat.binning(x, deal_column=fun_name,box_count=10, inplace=True))
# if not x.group_label.isna().all():
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.width', 999):
#         print(x)
# test.plot(only_binned=True)


alpha1
valid_right
alpha2
invalid
alpha3
invalid
alpha4
invalid
alpha5
valid_left
alpha6
valid_left
alpha7
sem_mid
alpha8
invalid
alpha9
invalid
alpha10
valid_right
alpha11
invalid
alpha12
valid_right
alpha13
valid_right
alpha14
valid_left
alpha15
invalid
alpha16
invalid
alpha17
valid_right
alpha18
invalid
alpha19
Empty DataFrame
Columns: [alpha19, ret_forward]
Index: []
errrrrrror

alpha20
invalid
alpha21
errrrrrrorlen(x) == 0

alpha22
invalid
alpha23
invalid
alpha24
invalid
alpha25
invalid
alpha26
valid_left
alpha27
errrrrrrorlen(x) == 0

alpha28
sem_mid
alpha29
valid_left
alpha30
valid_left
alpha31
invalid
alpha32
invalid
alpha33
valid_right
alpha34
valid_right
alpha35
invalid
alpha36
invalid
alpha37
invalid
alpha38
invalid
alpha39
Empty DataFrame
Columns: [alpha39, ret_forward]
Index: []
errrrrrror

alpha40
invalid
alpha41
valid_right
alpha42
valid_left
alpha43
valid_right
alpha44
invalid
alpha45
sem_mid
alpha46
sem_bilateral
alpha47
valid_left
alpha48
Empty DataFrame
Columns: [alp

In [593]:
ppp = inspect.signature(getattr(toys, 'alpha19')).parameters.keys()
aa = excute_for_multidates(data, lambda x: getattr(toys, 'alpha19')(*[x[param].copy() for param in ppp]) ,level=1)
aa.name = 'alpha19'
tt = FactorTest(target_field='alpha19')
tt.process_ind(pd.DataFrame(aa),data.ret_forward,simple=True)
tt.ind_ret_df

,,alpha19,ret_forward
date,code,,


In [624]:
ppp2 = inspect.signature(getattr(toys, 'alpha31')).parameters.keys()
aa2 = excute_for_multidates(data, lambda x: getattr(toys, 'alpha31')(*[x[param].copy() for param in ppp2]) ,level=1)
aa2.name = 'alpha31'
tt2 = FactorTest(target_field='alpha31')
tt2.process_ind(pd.DataFrame(aa2),data.ret_forward,simple=True)
tt2.get_ind_binned_ret_avg()
# tt2.ind_ret_df

group_label
1    -0.271883
2     0.706878
3     1.236217
4     0.433328
5    -1.290476
6     0.301921
7    -0.650514
8     0.390953
9    -0.930064
10   -1.057278
dtype: float64